In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import sklearn
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
df1 = pd.read_csv("C:\\Users\\Siddharth Kapoor\\Desktop\\ML Projects\\Gotham Cabs\\Train_New.csv")

In [3]:
df2 = df1[["duration","route","month","hour","weekday"]]
df2 = pd.get_dummies(df2,columns=["route"])
df2 = pd.get_dummies(df2,columns=["month"])
df2 = pd.get_dummies(df2,columns=["hour"])
df2 = pd.get_dummies(df2,columns=["weekday"])
df2["new"] = 1


In [4]:
#df2 = pd.get_dummies(df2,columns=["route"])
dfy = df2["duration"]
dfx = df2.drop(["duration"],axis =1)
#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()
#scaler.fit(dfx)
#X = scaler.transform(dfx)
from sklearn.cross_validation import train_test_split
dftrainx,dftestx,dftrainy,dftesty = train_test_split(dfx,dfy,train_size=.8,test_size=.2,random_state = 0)

c:\anaconda\envs\tensorflow\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
X_train = np.array(dftrainx)
y_train = np.array(dftrainy)
y_test = np.array(dftesty)
X_test = np.array(dftestx)

# Train Data Split

In [6]:
from sklearn.cross_validation import train_test_split
dftrainx1,dftrainx2,dftrainy1,dftrainy2 = train_test_split(X_train,y_train,train_size=.8,test_size=.2,random_state = 0)

# 1st Layer

# Linear Regression

In [12]:
from sklearn.linear_model import LinearRegression
regressor=LinearRegression()
regressor.fit(dftrainx1,dftrainy1)
y_pred_linear_1_layer = regressor.predict(dftrainx2)

In [14]:
mean_squared_error(dftrainy2,y_pred_linear_1_layer)

2.6221123948949956e+22

In [ ]:
y_pred_linear_1_layer_test = regressor.predict(dftestx)

# Decision Tree Regressor

In [15]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor(max_depth = 10)
tree_reg.fit(dftrainx1,dftrainy1)
y_pred_tree_1_layer = tree_reg.predict(dftrainx2)

In [16]:
mean_squared_error(dftrainy2,y_pred_tree_1_layer)

260307.31710202145

In [ ]:
y_pred_tree_1_layer_test = tree_reg.predict(dftestx)

# Lasso Regression

In [17]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha = 0.1)
lasso_reg.fit(dftrainx1,dftrainy1)
y_pred_lasso_1_layer = lasso_reg.predict(dftrainx2)

In [21]:
mean_squared_error(dftrainy2,y_pred_lasso_1_layer)

108642.1101732915

In [19]:
y_pred_lasso_1_layer_test = lasso_reg.predict(dftestx)

# Ridge Regression

In [20]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha = 2)
ridge_reg.fit(dftrainx1,dftrainy1)
y_pred_ridge_1_layer = ridge_reg.predict(dftrainx2)

In [22]:
mean_squared_error(dftrainy2,y_pred_ridge_1_layer)

99586.088603572192

In [23]:
y_pred_ridge_1_layer_test = ridge_reg.predict(dftestx)

# DNN Regressor

In [7]:
import tensorflow.contrib.learn as skflow
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

import numpy as np
np.set_printoptions(precision=2)

from sklearn.model_selection import train_test_split

#import matplotlib.pyplot as plt

#import logging
#logging.basicConfig(level=logging.INFO)
#logging.info('Tensorflow %s' % tf.__version__) # 1.4.1

# This is the magic function which the Deep Neural Network
# has to 'learn' (see http://neuralnetworksanddeeplearning.com/chap4.html)
f = lambda x: 0.2+0.4*x**2+0.3*x*np.sin(15*x)+0.05*np.cos(50*x)

# Generate the 'features'
X = np.linspace(0, 1, 1000001).reshape((-1, 1))

# Generate the 'labels'
y = f(X)

Instructions for updating:
Use the retry module or similar alternatives.


In [8]:
dftestx.shape,dftesty.shape

((258628, 699), (258628,))

In [9]:
def training_input_fn(batch_size=1):
    return tf.estimator.inputs.numpy_input_fn(x={'X': dftrainx1.astype(np.float32)},y=dftrainy1.astype(np.float32),batch_size=batch_size,num_epochs=None,shuffle=True)
#def training_input_fn2(batch_size=1):
#    return tf.estimator.inputs.numpy_input_fn(x={'X': dftrainx2.astype(np.float32)},y=dftrainy2.astype(np.float32),batch_size=batch_size,num_epochs=None,shuffle=True)

# for test

def test_input_fn():
    return tf.estimator.inputs.numpy_input_fn(x={'X': dftrainx2.astype(np.float32)},y=dftrainy2.astype(np.float32),num_epochs=1,shuffle=False)
def test_input_fn2():
    return tf.estimator.inputs.numpy_input_fn(x={'X': dftestx.astype(np.float32)},y=dftesty.astype(np.float32),num_epochs=1,shuffle=False)

In [10]:
# Network Design
# --------------
feature_columns = [tf.feature_column.numeric_column('X', shape=(699,))]

STEPS_PER_EPOCH = 100
EPOCHS = 200
BATCH_SIZE = 100

hidden_layers = [16, 16, 16, 16, 16]
dropout = 0.0

#MODEL_PATH='./DNNRegressors/'
#for hl in hidden_layers:
#    MODEL_PATH += '%s_' % hl
#MODEL_PATH += 'D0%s' % (int(dropout*10))
#logging.info('Saving to %s' % MODEL_PATH)


# Validation and Test Configuration
validation_metrics = {"MSE": tf.contrib.metrics.streaming_mean_squared_error}
test_config = skflow.RunConfig(save_checkpoints_steps=100,save_checkpoints_secs=None)

# Building the Network
regressorDNN = skflow.DNNRegressor(feature_columns=feature_columns,label_dimension=1,hidden_units=hidden_layers,model_dir=None,dropout=dropout,config=test_config)


In [11]:
for epoch in range(EPOCHS+1):
    # Fit the DNNRegressor (This is where the magic happens!!!)
    regressorDNN.fit(input_fn=training_input_fn(batch_size=BATCH_SIZE),steps=STEPS_PER_EPOCH)
    # Thats it -----------------------------
    # Start Tensorboard in Terminal:
    #tensorboard --logdir='./DNNRegressors/'
    # Now open Browser and visit localhost:6006\


    # This is just for fun and educational purpose:
    # Evaluate the DNNRegressor every 10th epoch
    if epoch%10==0:
        eval_dict = regressorDNN.evaluate(input_fn=test_input_fn(),metrics=validation_metrics)
        print('Epoch %i: %.5f MSE' % (epoch+1, eval_dict['MSE']))

Epoch 1: 157769.92188 MSE
Epoch 11: 94481.53125 MSE
Epoch 21: 89961.43750 MSE
Epoch 31: 88607.55469 MSE
Epoch 41: 88401.65625 MSE
Epoch 51: 87428.90625 MSE
Epoch 61: 87075.57812 MSE
Epoch 71: 86652.82031 MSE
Epoch 81: 86316.07812 MSE
Epoch 91: 86323.75781 MSE
Epoch 101: 85952.18750 MSE
Epoch 111: 85991.13281 MSE
Epoch 121: 86128.60156 MSE
Epoch 131: 85731.50000 MSE
Epoch 141: 85481.40625 MSE
Epoch 151: 85481.84375 MSE
Epoch 161: 85347.00000 MSE
Epoch 171: 85214.82031 MSE
Epoch 181: 85204.92188 MSE
Epoch 191: 85417.40625 MSE
Epoch 201: 85229.89844 MSE


In [24]:
y_pred_neural_1st_layer = regressorDNN.predict(x={'X': dftrainx2}, as_iterable=False)
y_pred_neural_2nd_layer = regressorDNN.predict(x={'X': X_test}, as_iterable=False)

In [ ]:
dftrainx2

In [ ]:
y_pred_tree_1_layer.shape,y_pred_neural_1st_layer.shape

# Combining and creating new training set

In [25]:
training_set_X = pd.DataFrame({'Lasso':y_pred_lasso_1_layer,'ridge':y_pred_ridge_1_layer,'Neural': y_pred_neural_1st_layer })
training_set_X["new"] =1

In [26]:
# Creating test set for 2nd layer

In [27]:
test_set_X = pd.DataFrame({'Lasso':y_pred_lasso_1_layer_test,'ridge':y_pred_ridge_1_layer_test,'Neural': y_pred_neural_2nd_layer})
test_set_X["new"] =1

# 2nd Layer
# XGBoost

In [28]:
import xgboost
xgb = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,colsample_bytree=1, max_depth=10)

In [29]:
xgb.fit(training_set_X,dftrainy2)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.08, max_delta_step=0,
       max_depth=10, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.75)

In [30]:
Y_xgb_predict = xgb.predict(test_set_X)
Y_xgb_predict

array([  445.31,   873.25,  1178.78, ...,   983.08,   299.72,   347.42], dtype=float32)

In [31]:
mean_squared_error(dftesty, Y_xgb_predict)

84671.494338393823

In [32]:
r2_score(dftesty, Y_xgb_predict)

0.71834569551051386

# Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
grbt = GradientBoostingRegressor(max_depth = 15, n_estimators=30, learning_rate =1.0)
grbt.fit(training_set_X,dftrainy2)

In [ ]:
y_Stacked_pred_grbt_ = grbt.predict(test_set_X)

In [ ]:
mean_squared_error(dftesty, y_Stacked_pred_grbt_)

In [ ]:
r2_score(dftesty, y_Stacked_pred_grbt_)

# Linear Regression

In [33]:
regressor_2_layer=LinearRegression()
regressor_2_layer.fit(training_set_X,dftrainy2)
y_pred_linear_2_layer = regressor_2_layer.predict(test_set_X)

In [34]:
mean_squared_error(dftesty, y_pred_linear_2_layer)

83837.20250214907

In [35]:
r2_score(dftesty, y_pred_linear_2_layer)

0.72112091388494881

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
model = RandomForestRegressor(n_jobs=-1)

In [ ]:
model.set_params(n_estimators=5)
model.fit(X_train, y_train)

In [ ]:
y_random_forest = model.predict(X_test)